In [1]:
import enchant
import scipy.stats
import numpy as np

In [2]:
lang = 'austronesian/tgl'

In [3]:
with open('../task0-data/DEVELOPMENT-LANGUAGES/'+lang+'.trn', encoding='utf8') as f:
    data = f.read().split('\n')
    data = [sent.split('\t') for sent in data]
    data.pop(-1)

In [4]:
data

[['patay', 'magpapatay', 'V;AGFOC;LGSPEC1'],
 ['tulak', 'itutulak', 'V;PFOC;LGSPEC1'],
 ['bagsak', 'magbabagsak', 'V;AGFOC;LGSPEC1'],
 ['dalaw', 'dumadalaw', 'V;IPFV;AGFOC'],
 ['ipon', 'ipon', 'V;NFIN'],
 ['gupit', 'ginugupitan', 'V;IPFV;PFOC'],
 ['sará', 'sará', 'V;NFIN'],
 ["sanlá'", "sinasanlá'", 'V;IPFV;PFOC'],
 ['kamay', 'kinamay', 'V;PFV;PFOC'],
 ['tanggap', 'tumatanggap', 'V;IPFV;AGFOC'],
 ['sigaw', 'sisigaw', 'V;AGFOC;LGSPEC1'],
 ['saká', 'saká', 'V;NFIN'],
 ['alaga', 'nag-aalaga', 'V;IPFV;AGFOC'],
 ['imbita', 'iniimbita', 'V;IPFV;PFOC'],
 ['akyat', 'iaakyat', 'V;PFOC;LGSPEC1'],
 ['tanggal', 'tatanggalin', 'V;PFOC;LGSPEC1'],
 ['kasal', 'magpapakasal', 'V;AGFOC;LGSPEC1'],
 ['kopya', 'kinokopya', 'V;IPFV;PFOC'],
 ['silip', 'silip', 'V;NFIN'],
 ['alaga', 'mag-aalaga', 'V;AGFOC;LGSPEC1'],
 ['bihis', 'magbibihis', 'V;AGFOC;LGSPEC1'],
 ['kíta', 'magkikíta', 'V;AGFOC;LGSPEC1'],
 ['tuloy', 'tutuluyan', 'V;PFOC;LGSPEC1'],
 ['ibig', 'umibig', 'V;PFV;AGFOC'],
 ['ábot', 'nag-aábot', 'V;IPF

In [5]:
dic = dict()
count = dict()
for sent in data:
    try :
        count[sent[0]]+=1
        dic[sent[0]].append( (sent[1],sent[2]) )
    except KeyError:
        try : 
            count[sent[0]] = 1
            dic[sent[0]] =  [ (sent[1],sent[2]) ,]
        except IndexError:
            raise IndexError(str(sent))

In [6]:
word = list(dic.keys())[2]
print('dating rating')
dic['dating rating']

dating rating


[('dumarating nararating', 'V;IPFV;AGFOC'),
 ('darating mararating', 'V;AGFOC;LGSPEC1'),
 ('dating rating', 'V;NFIN'),
 ('dumating narating', 'V;PFV;AGFOC')]

Rule 1 : the lemma determines the first the grammatical class (N,V,etc.)

In [7]:
# function to find the stem (longest common substring) from the string array
def findstem(arr):
    accept = False
    res = ""
    s = arr[0] # Take first word from array as reference

    for i in range(len(s)):
        for j in range(i + 1, len(s) + 1):

            # generating all possible substrings of our reference string
            stem = s[i:j]
            k = 1
            for k in range(1, len(arr)):
                accept = True
                if stem not in arr[k]:
                    accept = False
                    break

            # If current substring is present in all strings and its length is greater than current result
            if (accept and len(res) < len(stem)):
                res = stem

    return res

def stemDict(dic):
    stem_dic = dict()
    for lemma in list(dic.keys()):
        lemma_plus_form = [lemma] + [a[0] for a in dic[lemma]]
        stem = findstem(lemma_plus_form)
        start = lemma.index(stem)
        end = len(lemma) - (start+len(stem))
        pos = (start, end)

        try :
            stem_dic[pos].append(lemma)
        except KeyError:
            stem_dic[pos] = [lemma ,]
    return stem_dic

def stemDictToXY(dic):
    X = []
    Y = []
    for key in dic.keys():
        X += [[a] for a in dic[key]]
        Y += [key]*len(dic[key])
    return np.array(X), np.array(Y)

In [8]:
stem_dict = stemDict(dic)
x, y = stemDictToXY(stem_dict)
# print([(key, len(stem_dict[key])) for key in stem_dict.keys()])

In [9]:
def customMetric(x, y):
    return enchant.utils.levenshtein(x, y) / (len(x)+len(y))

class Knn:
    def __init__(self, n_neighbors):
        self.k = n_neighbors
        
    def fit(self, x, y):
        self.X = np.array(x)
        self.y = np.array(y)
     
    def predict(self, test):
        dists = np.zeros(len(self.X))
        for i in range(len(self.X)):
            dists[i] = customMetric(self.X[i], test)
        nn_ids = dists.argsort()[:self.k]
        return tuple(scipy.stats.mode(self.y[nn_ids])[0][0])

knn = Knn(n_neighbors=4)
knn.fit(x,y)
y_hat = knn.predict('liban')

In [10]:
gramm_classes = dict()
for lemma in dic.keys():
    gramm_classes[lemma] = dict()
    for form in dic[lemma]:
        gc = form[1].split(';')[0]
        try:
            gramm_classes[lemma][gc]+=1
        except KeyError:
            gramm_classes[lemma][gc]=1


In [11]:
#Number of exceptions
gramm_class_except = [lemma for lemma in gramm_classes.keys() if len(gramm_classes[lemma])>1]
print('Exceptions to Rule1 : ',len(gramm_class_except))
print('Nb of lemma : ', len(dic))
print('% of failure : ',round(len(gramm_class_except)/len(dic),4)*100)

Exceptions to Rule1 :  0
Nb of lemma :  344
% of failure :  0.0


In [12]:
with open('../task0-data/DEVELOPMENT-LANGUAGES/'+lang+'.tst', encoding='utf8') as f:
    test_data = f.read().split('\n')
    test_data = [sent.split('\t') for sent in test_data]
    test_data.pop(-1)

In [13]:
test_data

[['liban', 'V;PFOC;LGSPEC1'],
 ['isip', 'V;PFV;PFOC'],
 ['bagsak', 'V;NFIN'],
 ["alaga'", 'V;NFIN'],
 ['nakaw', 'V;IPFV;PFOC'],
 ['maneho', 'V;IPFV;PFOC'],
 ['másid', 'V;IPFV;PFOC'],
 ['taguyod', 'V;PFOC;LGSPEC1'],
 ["kuha'", 'V;PFOC;LGSPEC1'],
 ['basag', 'V;PFV;AGFOC'],
 ['hirap', 'V;NFIN'],
 ["babá'", 'V;NFIN'],
 ['panalo', 'V;IPFV;AGFOC'],
 ['hagod', 'V;PFOC;LGSPEC1'],
 ["sundó'", 'V;PFV;PFOC'],
 ['kusot', 'V;PFOC;LGSPEC1'],
 ['gayat', 'V;IPFV;PFOC'],
 ['bagsak', 'V;IPFV;PFOC'],
 ['suway', 'V;PFV;AGFOC'],
 ['ulit', 'V;IPFV;AGFOC'],
 ["sauli'", 'V;IPFV;AGFOC'],
 ['tabí', 'V;IPFV;AGFOC'],
 ['liban', 'V;PFV;PFOC'],
 ["babala'", 'V;IPFV;AGFOC'],
 ['senyas', 'V;PFV;AGFOC'],
 ["punó'", 'V;PFV;PFOC'],
 ["kuha'", 'V;AGFOC;LGSPEC1'],
 ['bili', 'V;PFV;AGFOC'],
 ['maneho', 'V;PFOC;LGSPEC1'],
 ['ingat', 'V;IPFV;AGFOC'],
 ['lungkot', 'V;NFIN'],
 ["arúga'", 'V;NFIN'],
 ['regalo', 'V;NFIN'],
 ['táas', 'V;PFV;AGFOC'],
 ['sákit', 'V;IPFV;PFOC'],
 ['kailangan', 'V;PFV;AGFOC'],
 ["gáwa'", 'V;IPFV;PFOC

In [14]:
test_dic = dict()
test_count = dict()
for sent in test_data:
    try :
        test_count[sent[0]]+=1
        test_dic[sent[0]].append( (sent[1]) )
    except KeyError:
        try : 
            test_count[sent[0]] = 1
            test_dic[sent[0]] =  [ (sent[1]) ,]
        except IndexError:
            raise IndexError(str(sent))

In [15]:
not_in_train = 0
train_lemmas_list = list(dic.keys())
for lemma in list(test_dic.keys()):
    if lemma in train_lemmas_list:
        not_in_train +=1

In [16]:
len(list(test_dic.keys()))

266

In [17]:
not_in_train

266

## -------------------------------------

In [18]:
import numpy as np
data_np = np.array(data)
morpho_attribute_raw = data_np[:,2]
morpho_attribute_splitted  = [attr.split(';') for attr in morpho_attribute_raw]       
morpho_attribute_flat = np.array([item for sublist in morpho_attribute_splitted for item in sublist])

list_morpho_attribute = np.unique(morpho_attribute_flat)
print(len(list_morpho_attribute), list_morpho_attribute)

# onehot_dict = {}
# for i, m in enumerate(list_morpho_attribute):
#     #vect = [0]*len(list_morpho_attribute)
#     vect = np.zeros(len(list_morpho_attribute))
#     vect[i] = 1
#     onehot_dict[m] = vect
    
# onehot_dict

7 ['AGFOC' 'IPFV' 'LGSPEC1' 'NFIN' 'PFOC' 'PFV' 'V']
